# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117443e+02     1.565882e+00
 * time: 0.08715987205505371
     1     1.056904e+01     9.546440e-01
 * time: 0.9019320011138916
     2    -1.138941e+01     1.031825e+00
 * time: 1.0481808185577393
     3    -3.397723e+01     7.296857e-01
 * time: 1.219925880432129
     4    -4.720753e+01     5.528416e-01
 * time: 1.3568170070648193
     5    -5.686766e+01     2.257707e-01
 * time: 1.5115947723388672
     6    -5.976740e+01     1.771394e-01
 * time: 1.6410009860992432
     7    -6.090766e+01     7.267951e-02
 * time: 1.7457737922668457
     8    -6.137642e+01     4.814259e-02
 * time: 1.8580679893493652
     9    -6.163567e+01     2.960500e-02
 * time: 1.9637649059295654
    10    -6.177862e+01     2.693879e-02
 * time: 2.075953960418701
    11    -6.191590e+01     2.089241e-02
 * time: 2.1812899112701416
    12    -6.200639e+01     1.972998e-02
 * time: 2.293677806854248
    13    -6.205294e+01     1.605363e-02
 * time: 2.399391889572

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557693
    AtomicNonlocal      14.8522660
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336822

    total               -62.261666459820
